In [2]:
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# ============================
# 1. Set Up Paths and Data Check
# ============================

# Define your base directory and file paths
base_dir = '/Users/bhaskarpramodchennupalli/Documents/Image_Detector'
csv_path = os.path.join(base_dir, 'train.csv')
train_data_dir = os.path.join(base_dir, 'train_data')

print("=== Stage 1: Data Check ===")
print("Starting data check...")

# Load CSV file containing file names and labels
df = pd.read_csv(csv_path)
print(f"Loaded CSV file: {csv_path} with {len(df)} records.")

# Drop an extra index column if it exists
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])
    print("Dropped extra index column 'Unnamed: 0'.")

# Check that every file in the CSV exists in the train_data directory
missing_files = []
for i, file_path in enumerate(df['file_name']):
    # The CSV file might list paths like "train_data/filename.jpg", so extract the file name.
    filename = os.path.basename(file_path)
    full_path = os.path.join(train_data_dir, filename)
    if not os.path.exists(full_path):
        missing_files.append(full_path)
    # Print progress every 100 files to avoid flooding output
    if i % 100 == 0:
        print(f"Checked {i} files...")

if missing_files:
    print(f"Total missing files: {len(missing_files)}")
    for f in missing_files:
        print("Missing:", f)
else:
    print("All files exist in the train_data directory!")
print("Data check complete.\n")

# ============================
# 2. Create a Custom Dataset Class
# ============================

print("=== Stage 2: Creating Custom Dataset ===")
print("Defining the CustomImageDataset class...")

class CustomImageDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        if 'Unnamed: 0' in self.data_frame.columns:
            self.data_frame = self.data_frame.drop(columns=['Unnamed: 0'])
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        # Get the image path and label from the CSV
        file_path = self.data_frame.iloc[idx]['file_name']
        # Extract just the file name in case the CSV includes a relative folder
        filename = os.path.basename(file_path)
        full_path = os.path.join(train_data_dir, filename)
        
        # Open the image and convert to RGB
        image = Image.open(full_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        
        # Get the label
        label = self.data_frame.iloc[idx]['label']
        return image, label

print("CustomImageDataset class defined.\n")

# ============================
# 3. Define Transformations, Dataset, and DataLoader
# ============================

print("=== Stage 3: Dataset & DataLoader Setup ===")
print("Setting up image transformations...")

# Define the image transformations (resize, convert to tensor)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 pixels
    transforms.ToTensor(),
    # Optionally, add normalization:
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
print("Transformations defined.")

print("Creating the dataset...")
train_dataset = CustomImageDataset(csv_file=csv_path, transform=transform)
print(f"Dataset created with {len(train_dataset)} images.")

print("Creating the DataLoader...")
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
print(f"DataLoader created with batch size {batch_size}.\n")

# ============================
# 4. Define a Simple CNN Model
# ============================

print("=== Stage 4: Model Definition ===")
print("Defining the SimpleCNN model...")

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        # For an input image size of 224x224, after a single max-pool (kernel=2),
        # the spatial dimensions become 112x112.
        self.fc_layer = nn.Sequential(
            nn.Linear(16 * 112 * 112, 64),  # Adjust dimensions based on your conv output
            nn.ReLU(),
            nn.Linear(64, 1)  # Single output for binary classification
        )
    
    def forward(self, x):
        x = self.conv_layer(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.fc_layer(x)
        return x

print("Instantiating the CNN model...")
model = SimpleCNN()
print("CNN model defined and instantiated.\n")

# ============================
# 5. Define Loss Function and Optimizer
# ============================

print("=== Stage 5: Loss & Optimizer Setup ===")
print("Setting up loss function and optimizer...")

criterion = nn.BCEWithLogitsLoss()  # For binary classification (0 or 1)
optimizer = optim.Adam(model.parameters(), lr=0.001)
print("Loss function and optimizer are set.\n")

# ============================
# 6. Training Loop
# ============================

print("=== Stage 6: Training ===")
num_epochs = 10
print("Starting training...\n")

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0
    print(f"Epoch {epoch+1}/{num_epochs} started.")
    for batch_idx, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()           # Reset gradients
        outputs = model(images)         # Forward pass
        labels = labels.float().unsqueeze(1)  # Convert labels to float and shape (batch_size, 1)
        loss = criterion(outputs, labels)
        loss.backward()                 # Backward pass
        optimizer.step()                # Update model parameters
        
        epoch_loss += loss.item() * images.size(0)
        
        if batch_idx % 10 == 0:
            print(f"Epoch {epoch+1}, Batch {batch_idx}: Loss = {loss.item():.4f}")
    
    avg_loss = epoch_loss / len(train_dataset)
    print(f"Epoch {epoch+1} completed. Average Loss: {avg_loss:.4f}\n")

print("Training complete.\n")


=== Stage 1: Data Check ===
Starting data check...
Loaded CSV file: /Users/bhaskarpramodchennupalli/Documents/Image_Detector/train.csv with 79950 records.
Dropped extra index column 'Unnamed: 0'.
Checked 0 files...
Checked 100 files...
Checked 200 files...
Checked 300 files...
Checked 400 files...
Checked 500 files...
Checked 600 files...
Checked 700 files...
Checked 800 files...
Checked 900 files...
Checked 1000 files...
Checked 1100 files...
Checked 1200 files...
Checked 1300 files...
Checked 1400 files...
Checked 1500 files...
Checked 1600 files...
Checked 1700 files...
Checked 1800 files...
Checked 1900 files...
Checked 2000 files...
Checked 2100 files...
Checked 2200 files...
Checked 2300 files...
Checked 2400 files...
Checked 2500 files...
Checked 2600 files...
Checked 2700 files...
Checked 2800 files...
Checked 2900 files...
Checked 3000 files...
Checked 3100 files...
Checked 3200 files...
Checked 3300 files...
Checked 3400 files...
Checked 3500 files...
Checked 3600 files...
Ch

Checked 58200 files...
Checked 58300 files...
Checked 58400 files...
Checked 58500 files...
Checked 58600 files...
Checked 58700 files...
Checked 58800 files...
Checked 58900 files...
Checked 59000 files...
Checked 59100 files...
Checked 59200 files...
Checked 59300 files...
Checked 59400 files...
Checked 59500 files...
Checked 59600 files...
Checked 59700 files...
Checked 59800 files...
Checked 59900 files...
Checked 60000 files...
Checked 60100 files...
Checked 60200 files...
Checked 60300 files...
Checked 60400 files...
Checked 60500 files...
Checked 60600 files...
Checked 60700 files...
Checked 60800 files...
Checked 60900 files...
Checked 61000 files...
Checked 61100 files...
Checked 61200 files...
Checked 61300 files...
Checked 61400 files...
Checked 61500 files...
Checked 61600 files...
Checked 61700 files...
Checked 61800 files...
Checked 61900 files...
Checked 62000 files...
Checked 62100 files...
Checked 62200 files...
Checked 62300 files...
Checked 62400 files...
Checked 625

Epoch 1, Batch 730: Loss = 0.3997
Epoch 1, Batch 740: Loss = 0.2101
Epoch 1, Batch 750: Loss = 0.3443
Epoch 1, Batch 760: Loss = 0.4740
Epoch 1, Batch 770: Loss = 0.2801
Epoch 1, Batch 780: Loss = 0.3921
Epoch 1, Batch 790: Loss = 0.3225
Epoch 1, Batch 800: Loss = 0.3194
Epoch 1, Batch 810: Loss = 0.1609
Epoch 1, Batch 820: Loss = 0.2226
Epoch 1, Batch 830: Loss = 0.2991
Epoch 1, Batch 840: Loss = 0.2308
Epoch 1, Batch 850: Loss = 0.4792
Epoch 1, Batch 860: Loss = 0.1404
Epoch 1, Batch 870: Loss = 0.1947
Epoch 1, Batch 880: Loss = 0.2094
Epoch 1, Batch 890: Loss = 0.2944
Epoch 1, Batch 900: Loss = 0.4648
Epoch 1, Batch 910: Loss = 0.2063
Epoch 1, Batch 920: Loss = 0.2277
Epoch 1, Batch 930: Loss = 0.4092
Epoch 1, Batch 940: Loss = 0.1587
Epoch 1, Batch 950: Loss = 0.3047
Epoch 1, Batch 960: Loss = 0.3674
Epoch 1, Batch 970: Loss = 0.1911
Epoch 1, Batch 980: Loss = 0.2931
Epoch 1, Batch 990: Loss = 0.4321
Epoch 1, Batch 1000: Loss = 0.2819
Epoch 1, Batch 1010: Loss = 0.2501
Epoch 1, Bat

Epoch 2, Batch 590: Loss = 0.2528
Epoch 2, Batch 600: Loss = 0.0485
Epoch 2, Batch 610: Loss = 0.1527
Epoch 2, Batch 620: Loss = 0.1008
Epoch 2, Batch 630: Loss = 0.1003
Epoch 2, Batch 640: Loss = 0.2359
Epoch 2, Batch 650: Loss = 0.1040
Epoch 2, Batch 660: Loss = 0.1708
Epoch 2, Batch 670: Loss = 0.2222
Epoch 2, Batch 680: Loss = 0.0664
Epoch 2, Batch 690: Loss = 0.2546
Epoch 2, Batch 700: Loss = 0.1379
Epoch 2, Batch 710: Loss = 0.1293
Epoch 2, Batch 720: Loss = 0.2812
Epoch 2, Batch 730: Loss = 0.2603
Epoch 2, Batch 740: Loss = 0.1034
Epoch 2, Batch 750: Loss = 0.0692
Epoch 2, Batch 760: Loss = 0.0862
Epoch 2, Batch 770: Loss = 0.3932
Epoch 2, Batch 780: Loss = 0.2144
Epoch 2, Batch 790: Loss = 0.0924
Epoch 2, Batch 800: Loss = 0.1960
Epoch 2, Batch 810: Loss = 0.0389
Epoch 2, Batch 820: Loss = 0.3242
Epoch 2, Batch 830: Loss = 0.0441
Epoch 2, Batch 840: Loss = 0.1007
Epoch 2, Batch 850: Loss = 0.2220
Epoch 2, Batch 860: Loss = 0.1413
Epoch 2, Batch 870: Loss = 0.2264
Epoch 2, Batch

Epoch 3, Batch 450: Loss = 0.0957
Epoch 3, Batch 460: Loss = 0.1426
Epoch 3, Batch 470: Loss = 0.1394
Epoch 3, Batch 480: Loss = 0.1403
Epoch 3, Batch 490: Loss = 0.0212
Epoch 3, Batch 500: Loss = 0.0886
Epoch 3, Batch 510: Loss = 0.1655
Epoch 3, Batch 520: Loss = 0.1918
Epoch 3, Batch 530: Loss = 0.3057
Epoch 3, Batch 540: Loss = 0.1699
Epoch 3, Batch 550: Loss = 0.0913
Epoch 3, Batch 560: Loss = 0.0301
Epoch 3, Batch 570: Loss = 0.0793
Epoch 3, Batch 580: Loss = 0.0652
Epoch 3, Batch 590: Loss = 0.0687
Epoch 3, Batch 600: Loss = 0.1489
Epoch 3, Batch 610: Loss = 0.0644
Epoch 3, Batch 620: Loss = 0.0894
Epoch 3, Batch 630: Loss = 0.1743
Epoch 3, Batch 640: Loss = 0.0832
Epoch 3, Batch 650: Loss = 0.2896
Epoch 3, Batch 660: Loss = 0.0509
Epoch 3, Batch 670: Loss = 0.1725
Epoch 3, Batch 680: Loss = 0.0792
Epoch 3, Batch 690: Loss = 0.1262
Epoch 3, Batch 700: Loss = 0.0407
Epoch 3, Batch 710: Loss = 0.0823
Epoch 3, Batch 720: Loss = 0.0855
Epoch 3, Batch 730: Loss = 0.2533
Epoch 3, Batch

...

Epoch 6, Batch 2400: Loss = 0.0151
Epoch 6, Batch 2410: Loss = 0.0137
Epoch 6, Batch 2420: Loss = 0.1400
Epoch 6, Batch 2430: Loss = 0.0520
Epoch 6, Batch 2440: Loss = 0.0487
Epoch 6, Batch 2450: Loss = 0.0130
Epoch 6, Batch 2460: Loss = 0.2015
Epoch 6, Batch 2470: Loss = 0.0219
Epoch 6, Batch 2480: Loss = 0.0237
Epoch 6, Batch 2490: Loss = 0.1021
Epoch 6 completed. Average Loss: 0.0447

Epoch 7/10 started.
Epoch 7, Batch 0: Loss = 0.0958
Epoch 7, Batch 10: Loss = 0.0094
Epoch 7, Batch 20: Loss = 0.0046
Epoch 7, Batch 30: Loss = 0.0153
Epoch 7, Batch 40: Loss = 0.0832
Epoch 7, Batch 50: Loss = 0.2353
Epoch 7, Batch 60: Loss = 0.0030
Epoch 7, Batch 70: Loss = 0.0363
Epoch 7, Batch 80: Loss = 0.0376
Epoch 7, Batch 90: Loss = 0.0270
Epoch 7, Batch 100: Loss = 0.0531
Epoch 7, Batch 110: Loss = 0.0001
Epoch 7, Batch 120: Loss = 0.0198
Epoch 7, Batch 130: Loss = 0.0924
Epoch 7, Batch 140: Loss = 0.0688
Epoch 7, Batch 150: Loss = 0.0062
Epoch 7, Batch 160: Loss = 0.0043
Epoch 7, Batch 170: Lo

Epoch 7, Batch 2260: Loss = 0.0267
Epoch 7, Batch 2270: Loss = 0.0321
Epoch 7, Batch 2280: Loss = 0.0129
Epoch 7, Batch 2290: Loss = 0.0045
Epoch 7, Batch 2300: Loss = 0.0089
Epoch 7, Batch 2310: Loss = 0.1140
Epoch 7, Batch 2320: Loss = 0.0160
Epoch 7, Batch 2330: Loss = 0.0540
Epoch 7, Batch 2340: Loss = 0.0416
Epoch 7, Batch 2350: Loss = 0.0438
Epoch 7, Batch 2360: Loss = 0.0168
Epoch 7, Batch 2370: Loss = 0.0372
Epoch 7, Batch 2380: Loss = 0.0008
Epoch 7, Batch 2390: Loss = 0.0431
Epoch 7, Batch 2400: Loss = 0.1341
Epoch 7, Batch 2410: Loss = 0.0117
Epoch 7, Batch 2420: Loss = 0.0080
Epoch 7, Batch 2430: Loss = 0.0151
Epoch 7, Batch 2440: Loss = 0.0326
Epoch 7, Batch 2450: Loss = 0.0031
Epoch 7, Batch 2460: Loss = 0.0243
Epoch 7, Batch 2470: Loss = 0.1910
Epoch 7, Batch 2480: Loss = 0.0024
Epoch 7, Batch 2490: Loss = 0.0103
Epoch 7 completed. Average Loss: 0.0337

Epoch 8/10 started.
Epoch 8, Batch 0: Loss = 0.0049
Epoch 8, Batch 10: Loss = 0.0018
Epoch 8, Batch 20: Loss = 0.0392
E

Epoch 8, Batch 2120: Loss = 0.1036
Epoch 8, Batch 2130: Loss = 0.0060
Epoch 8, Batch 2140: Loss = 0.0051
Epoch 8, Batch 2150: Loss = 0.0755
Epoch 8, Batch 2160: Loss = 0.0253
Epoch 8, Batch 2170: Loss = 0.0007
Epoch 8, Batch 2180: Loss = 0.0513
Epoch 8, Batch 2190: Loss = 0.0036
Epoch 8, Batch 2200: Loss = 0.0010
Epoch 8, Batch 2210: Loss = 0.0132
Epoch 8, Batch 2220: Loss = 0.0134
Epoch 8, Batch 2230: Loss = 0.0111
Epoch 8, Batch 2240: Loss = 0.0651
Epoch 8, Batch 2250: Loss = 0.0196
Epoch 8, Batch 2260: Loss = 0.0128
Epoch 8, Batch 2270: Loss = 0.0081
Epoch 8, Batch 2280: Loss = 0.0269
Epoch 8, Batch 2290: Loss = 0.0011
Epoch 8, Batch 2300: Loss = 0.0022
Epoch 8, Batch 2310: Loss = 0.0009
Epoch 8, Batch 2320: Loss = 0.0066
Epoch 8, Batch 2330: Loss = 0.0057
Epoch 8, Batch 2340: Loss = 0.0227
Epoch 8, Batch 2350: Loss = 0.0069
Epoch 8, Batch 2360: Loss = 0.0118
Epoch 8, Batch 2370: Loss = 0.0073
Epoch 8, Batch 2380: Loss = 0.0008
Epoch 8, Batch 2390: Loss = 0.0193
Epoch 8, Batch 2400:

Epoch 9, Batch 1980: Loss = 0.0008
Epoch 9, Batch 1990: Loss = 0.0671
Epoch 9, Batch 2000: Loss = 0.0081
Epoch 9, Batch 2010: Loss = 0.0600
Epoch 9, Batch 2020: Loss = 0.0350
Epoch 9, Batch 2030: Loss = 0.0009
Epoch 9, Batch 2040: Loss = 0.0060
Epoch 9, Batch 2050: Loss = 0.0532
Epoch 9, Batch 2060: Loss = 0.0218
Epoch 9, Batch 2070: Loss = 0.1481
Epoch 9, Batch 2080: Loss = 0.0724
Epoch 9, Batch 2090: Loss = 0.0554
Epoch 9, Batch 2100: Loss = 0.0954
Epoch 9, Batch 2110: Loss = 0.0081
Epoch 9, Batch 2120: Loss = 0.0018
Epoch 9, Batch 2130: Loss = 0.0210
Epoch 9, Batch 2140: Loss = 0.0814
Epoch 9, Batch 2150: Loss = 0.0117
Epoch 9, Batch 2160: Loss = 0.0204
Epoch 9, Batch 2170: Loss = 0.3897
Epoch 9, Batch 2180: Loss = 0.0372
Epoch 9, Batch 2190: Loss = 0.0052
Epoch 9, Batch 2200: Loss = 0.0101
Epoch 9, Batch 2210: Loss = 0.0027
Epoch 9, Batch 2220: Loss = 0.0967
Epoch 9, Batch 2230: Loss = 0.0483
Epoch 9, Batch 2240: Loss = 0.0441
Epoch 9, Batch 2250: Loss = 0.0020
Epoch 9, Batch 2260:

Epoch 10, Batch 1790: Loss = 0.0028
Epoch 10, Batch 1800: Loss = 0.0008
Epoch 10, Batch 1810: Loss = 0.0005
Epoch 10, Batch 1820: Loss = 0.0007
Epoch 10, Batch 1830: Loss = 0.0293
Epoch 10, Batch 1840: Loss = 0.0065
Epoch 10, Batch 1850: Loss = 0.0573
Epoch 10, Batch 1860: Loss = 0.1259
Epoch 10, Batch 1870: Loss = 0.0146
Epoch 10, Batch 1880: Loss = 0.0009
Epoch 10, Batch 1890: Loss = 0.0180
Epoch 10, Batch 1900: Loss = 0.0146
Epoch 10, Batch 1910: Loss = 0.0082
Epoch 10, Batch 1920: Loss = 0.0021
Epoch 10, Batch 1930: Loss = 0.0071
Epoch 10, Batch 1940: Loss = 0.0154
Epoch 10, Batch 1950: Loss = 0.1904
Epoch 10, Batch 1960: Loss = 0.0287
Epoch 10, Batch 1970: Loss = 0.0078
Epoch 10, Batch 1980: Loss = 0.0108
Epoch 10, Batch 1990: Loss = 0.0043
Epoch 10, Batch 2000: Loss = 0.0106
Epoch 10, Batch 2010: Loss = 0.0129
Epoch 10, Batch 2020: Loss = 0.0048
Epoch 10, Batch 2030: Loss = 0.0136
Epoch 10, Batch 2040: Loss = 0.0158
Epoch 10, Batch 2050: Loss = 0.0040
Epoch 10, Batch 2060: Loss =

In [10]:
#Save Model

In [11]:
# Add this after training completes
torch.save(model.state_dict(), os.path.join(base_dir, 'model.pth'))
print("Model saved as model.pth")

Model saved as model.pth
